In [ ]:
import numpy
from urllib.request import urlopen
import scipy.optimize
import random
from math import exp
from math import log
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def parseData(fname):
    for l in urlopen(fname):
        yield eval(l)

print("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print("done")

def feature(datum):
    feat = [1, datum['review/taste'], datum['review/appearance'], datum['review/aroma'], datum['review/palate'], datum['review/overall']]
    return feat

X = [feature(d) for d in data]
y = [d['beer/ABV'] >= 6.5 for d in data]

def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

##################################################
# Logistic regression by gradient ascent         #
##################################################

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    fracPos = sum(y) / len(y)
    fracNeg = 1 - fracPos
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        if y[i]:
      #loglikelihood -= 0.5/fracPos * log(1 + exp(-logit)) # balanced
          loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
      #loglikelihood -= 0.5/fracNeg * (log(1 + exp(-logit)) + logit) # balanced
          loglikelihood -= (log(1 + exp(-logit)) + logit)
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
    return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    fracPos = sum(y) / len(y)
    fracNeg = 1 - fracPos
    dl = [0]*len(theta)
    for i in range(len(X)):
        logit = inner(X[i], theta)
        for k in range(len(theta)):
            if y[i]:
        #dl[k] += 0.5/fracPos * (X[i][k] * (1 - sigmoid(logit))) # balanced
            dl[k] += X[i][k] * (1 - sigmoid(logit))
            if not y[i]:
        #dl[k] += 0.5/fracNeg * (X[i][k] * (1 - sigmoid(logit)) - X[i][k]) # balanced
           dl[k] += X[i][k] * (1 - sigmoid(logit)) - X[i][k]
    for k in range(len(theta)):
        dl[k] -= lam*2*theta[k]
    return numpy.array([-x for x in dl])

##################################################
# Train/validation/test splits                   #
##################################################

X_train = X[:int(len(X)/3)]
y_train = y[:int(len(y)/3)]
X_validate = X[int(len(X)/3):int(2*len(X)/3)]
y_validate = y[int(len(y)/3):int(2*len(y)/3)]
X_test = X[int(2*len(X)/3):]
y_test = y[int(2*len(X)/3):]

##################################################
# Train                                          #
##################################################

def train(lam):
    theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, pgtol = 10, args = (X_train, y_train, lam))
    return theta

##################################################
# Predict                                        #
##################################################

def performance(theta, y_train, y_validate, y_test, X_train, X_validate, X_test):
                      kkkk
                      scores_train = [inner(theta,x) for x in X_train]
                      scores_validate = [inner(theta,x) for x in X_validate]
                      scores_test = [inner(theta,x) for x in X_test]
                      predictions_train = [s > 0 for s in scores_train]
                      predictions_validate = [s > 0 for s in scores_validate]
                      predictions_test = [s > 0 for s in scores_test]
                      correct_train = [(a==b) for (a,b) in zip(predictions_train,y_train)]
                      correct_validate = [(a==b) for (a,b) in zip(predictions_validate,y_validate)]
                      correct_test = [(a==b) for (a,b) in zip(predictions_test,y_test)]
                      acc_train = sum(correct_train) * 1.0 / len(correct_train)
                      acc_validate = sum(correct_validate) * 1.0 / len(correct_validate)
                      acc_test = sum(correct_test) * 1.0 / len(correct_test)
                      TP = sum([(a and b) for (a,b) in zip(predictions_test, y_test)])
                      TN = sum([(not a and not b) for (a,b) in zip(predictions_test, y_test)])
                      FP = sum([(a and not b) for (a,b) in zip(predictions_test, y_test)])
                      FN = sum([(not a and b) for (a,b) in zip(predictions_test, y_test)])
                      TPR = TP / (TP + FN)
                      TNR = TN / (TN + FP)
                      BER = 1.0 - 0.5*(TPR + TNR)
                      print("BER = " + str(BER))
                      labelsSortedByScore = list(zip(scores_test, y_test))
                      labelsSortedByScore.sort()
                      labelsSortedByScore.reverse()
                      labelsSortedByScore = [a[1] for a in labelsSortedByScore]
                      return acc_train, acc_validate, acc_test, (TP, TN, FP, FN, BER), labelsSortedByScore

##################################################
# Train/validation/test                          #
##################################################

lam = 1.0
theta = train(lam)
acc_train, acc_validate, acc_test, _, _ = performance(theta, y_train, y_validate, y_test, X_train, X_validate, X_test)
print("lambda = " + str(lam) + ";\ttrain=" + str(acc_train) + "; validate=" + str(acc_validate) + "; test=" + str(acc_test))

##################################################
# Better features                                #
##################################################

words = ["lactic", "tart", "sour", "citric", "sweet", "acid", "hop", "fruit", "salt", "spicy"]

def feature(datum):
  text = datum['review/text'].lower()
  feat = [1]
  for w in words:
    feat.append(text.count(w))
  return feat

X = [feature(d) for d in data]
X_train = X[:int(len(X)/3)]
X_validate = X[int(len(X)/3):int(2*len(X)/3)]
X_test = X[int(2*len(X)/3):]

theta = train(lam)
acc_train, acc_validate, acc_test, _, _ = performance(theta, y_train, y_validate, y_test, X_train, X_validate, X_test)
print("lambda = " + str(lam) + ";\ttrain=" + str(acc_train) + "; validate=" + str(acc_validate) + "; test=" + str(acc_test))

##################################################
# Error rates                                    #
##################################################

_, _, _, (TP, TN, FP, FN, BER), labelsSortedByScore = performance(theta, y_train, y_validate, y_test, X_train, X_validate, X_test)

print("TP = ", TP)
print("TN = ", TN)
print("FP = ", FP)
print("FN = ", FN)
print("BER = ", BER)

##################################################
# Validation pipeline                            #
##################################################

bestLambda = None
bestLambdaAcc = None
bestValidate = None

for lam in [0, 0.01, 1.0, 100.0]:
  theta = train(lam)
  acc_train, acc_validate, acc_test, _, _ = performance(theta, y_train, y_validate, y_test, X_train, X_validate, X_test)
  if (not bestLambda) or acc_validate > bestValidate:
    bestLambda = lam
    bestValidate = acc_validate
    bestLambdaAcc = (acc_train, acc_validate, acc_test)
  print("lambda = " + str(lam) + ";\ttrain=" + str(acc_train) + "; validate=" + str(acc_validate) + "; test=" + str(acc_test))

print("Best lambda on validation set = " + str(lam))
print("train/valid/test accuracy = " + str(bestLambdaAcc))

##################################################
# PCA                                            #
##################################################

Xn = [x[1:] for x in X_train] # Drop the offset (shouldn't make a significant difference to answers)
Xn = numpy.matrix(Xn)

pca = PCA()
pca.fit(Xn)

print(pca.components_)

##################################################
# Replace points by their mean                   #
##################################################

print("Reconstruction error when replacing points by the mean of the corresponding coordinate:")
print(numpy.linalg.norm(Xn - Xn.mean(0))**2) 

##################################################
# Reconstruction error with two dimensions      #
##################################################

Yn = Xn*pca.components_.T

print("First transformed data point " + str(Yn[0]))

yVar = numpy.var(Yn,0)
print("Reconstruction error in the new basis:")
print(len(Yn) * sum(yVar.tolist()[0][2:])) # Reconstruction error

##################################################
# Plot the PCA dimensions                        #
##################################################

plt.scatter([a[0,0] for a in Yn], [a[0,1] for a in Yn], color='r')
Yn_IPA = [Yn[i] for i in range(len(Yn)) if y[i]]
plt.scatter([a[0,0] for a in Yn_IPA], [a[0,1] for a in Yn_IPA], color='b')
plt.show()